In [124]:
# imports
import pandas as pd
import plotly.graph_objects as go

In [125]:
# Read in data
df = pd.read_excel('2022-03-17 Graduates MDSE - current jobs.xlsx')

# Correct columns
new_columns = ['snr', 'employer', 'role', 'industry', 'organization_type', 'job_title']
df = df.set_axis(new_columns, axis = 1)

# Drop NaN
df = df.dropna()

# Change 'Government' since it is redundant
df.loc[df["organization_type"] == "Government", "organization_type"] = "Government, municipalities and non-profit"

# Print data
df

,snr,employer,role,industry,organization_type,job_title
0,2007400,Floryn,Data Scientist,Finance,Start-up/Scale-up,Entrepreneurial Data Analyst
1,1250535,vd Vleuten Elektrotechniek & Beveiliging BV,Allround Employee,Building and Construction,SME,Other
2,2007307,ABN AMRO Bank N.V.,Data Scientist,Finance,Large corporates & multinationals,Entrepreneurial Data Analyst
3,2005986,Bain & Company,Manager Data Science,Consulting,Large corporates & multinationals,Data Consultant/Entrepreneur
4,2007450,PostNL,Data Scientist,Public governance,Large corporates & multinationals,Entrepreneurial Data Analyst
...,...,...,...,...,...,...
216,2047057,Ahti,Junior Data Scientist,Healthcare,"Government, municipalities and non-profit",Entrepreneurial Data Analyst
218,2048198,Data Scientist,Itility,Information tech. & services,Large corporates & multinationals,Entrepreneurial Data Analyst
219,2045585,Data Science Intern,European Central Bank,Finance,"Government, municipalities and non-profit",Entrepreneurial Data Analyst
221,2043054,Eurofiber Nederland,Data Analyst,Information Tech. & Services,SME,Entrepreneurial Data Analyst


In [126]:
# Data analysis
for column in range(1, len(df.columns)):
    uniques_count = len(df.iloc[:,column].unique())
    uniques = df.iloc[:,column].unique()
    
    print("=====For the column '{0}'=====\n".format(df.columns[column]))
    print("uniques count: {0}\nuniques: {1}\n\n".format(uniques_count, uniques))

=====For the column 'employer'=====

uniques count: 153
uniques: ['Floryn' 'vd Vleuten Elektrotechniek & Beveiliging BV'
 'ABN AMRO Bank N.V.' 'Bain & Company' 'PostNL' 'Viqtor Davis'
 'Digital Power' 'Sendcloud' 'Bundesdruckerei' 'LionsTale' 'Nippur'
 'Enjins' 'Channext' 'Koninklijke Luchtvaart Maatschappij(KLM)' '3D HUBS'
 'Nextgem' 'Heijmans' 'IKNL (Integraal Kankercentrum Nederland)' 'Ilionx'
 'Sogeti' 'Cognizant' 'Funda' 'GoDataDriven' 'Prosperty'
 'JADS MKB Datalab' 'Techonomy' 'Admolly' 'Ikbenfrits'
 'JADS/Tilburg University' 'Accenture' 'Carepay'
 'Uitgetypt.nl/MSdVServices' 'Rabobank' 'Bluetick' 'Wartsila Netherlands'
 'Auditdienst Rijk' 'Via Traffic Solutions Software' 'Quooker B.V.'
 'Eneco' 'Rijk Zwaan' 'Vartion' 'Kinetic Analysis' 'ORTEC B.V.'
 'DPG Media Nederland' 'IG&H Consulting & Interim' 'Beet' 'Marketredesign'
 'Visionairy' 'Valcon / Intense'
 'Koninklijk Nederlands Meteorologisch Instituut (KNMI)' 'Lynxx'
 'Eindhoven University of Technology'
 'e-Luscious B2C Wine,

In [127]:
# transform df into a source-target pair for Sankey diagram

# for every job_title, grab every organization type + count
foo_1 = df.groupby(['job_title', 'organization_type']).snr.count().to_frame()
foo_1.reset_index(inplace=True)
foo_1.columns = ['source', 'target', 'value']


# for every organization type, grab every industry + count
foo_2 = df.groupby(['organization_type', 'industry']).snr.count().to_frame()
foo_2.reset_index(inplace=True)
foo_2.columns = ['source', 'target', 'value']

# paste them together
df_prepped = pd.concat([foo_1, foo_2])

In [128]:
# Now apparently it needs to be split in 2..

# nodes
# {ID, label, colour}
label = []
label.append(df.organization_type.unique())
label.append(df.job_title.unique())
label.append(df.industry.unique())
nodes = pd.DataFrame([item for sub_list in label for item in sub_list]) # flatten list
nodes = nodes.assign(ID = range(0,len(nodes.iloc[:,0]))) # create ID column
nodes = nodes.assign(colour = ["#424E51"]*len(nodes.iloc[:,0]))
nodes.columns = ["label", "ID", "colour"]

In [132]:
# links
# {source, target, value, link_colour}
links = df_prepped.copy() # deep copy to prevent reference changes
mapping = dict(zip(nodes.label, nodes.ID)) # maps labels to ID's
links['source'] = df_prepped['source'].map(mapping)
links['target'] = df_prepped['target'].map(mapping)
links = links.assign(colour = ["#E5DCD6"]*len(links.iloc[:,0])) # add colour column

links

,source,target,value,colour
0,8,5,4,#E5DCD6
1,8,2,16,#E5DCD6
2,8,1,11,#E5DCD6
3,8,0,24,#E5DCD6
4,9,3,2,#E5DCD6
...,...,...,...,...
39,0,17,2,#E5DCD6
40,4,11,1,#E5DCD6
41,4,15,1,#E5DCD6
42,4,14,1,#E5DCD6


In [133]:
# create plot

fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = nodes.label,
      color = nodes.colour
    ),
    link = dict(
      source = links.source,
      target = links.target,
      value = links.value
  ))])

fig.update_layout(title_text="Basic Sankey Diagram", font_size=10)
fig.show()